In [69]:
from fake_useragent import UserAgent

import pandas as pd
import json
import requests

In [21]:
us = UserAgent()
while True:
    if 'Chrome' in us.random:
        user_string = us.random
        print(user_string)
        break

hdr = {'User-Agent':f'{user_string}'}

page_cnt = 100 # 총 5490 페이지까지 있음 || page_cnt는 알아서 조절해서 사용할 것 (원하는 페이지의 수만큼 조절)
count = 1

url_list = []

for c in range(1, page_cnt):
    response = requests.get(f"https://www.10000recipe.com/recipe/list.html?order=reco&page={page_cnt}", headers=hdr)
    if response.status_code == 200:
        soup = BS(response.text, 'html.parser')
        for a_href in soup.find_all(class_='common_sp_link'):
            url_list.append("https://www.10000recipe.com"+a_href['href'])
            
    print(count)
    count += 1

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0 Viewer/99.9.9009.89
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [23]:
recipes = []

for url in new_urls:
    response = requests.get(f"{url}", headers=hdr)

    if response.status_code == 200:
        try:
            soup = BS(response.text, 'html.parser')
            food_info = soup.find(attrs={'type':'application/ld+json'})

            json_result = json.loads(food_info.text)
            ingredient = ','.join(json_result['recipeIngredient'])
            name = json_result['name']

            recipe = [json_result['recipeInstructions'][i]['text'] for i in range(len(json_result['recipeInstructions']))]
            for i in range(len(recipe)):
                recipe[i] = f'{i+1}. ' + recipe[i]

            res = {
                'name': name,
                'ingredients': ingredient,
                'recipe': recipe
            }

            recipes.append(res)
        except Exception as ex:
            pass
        
with open('recipes.json', 'w', encoding='utf-8') as f:
    json.dump(recipes, f, ensure_ascii=False, indent=4)

In [75]:
df = pd.DataFrame(recipes)
df['recipe'] = df['recipe'].apply(lambda x: '\n'.join(x))

print(df.shape)
print(df.info())

df.head(10)

(8, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         8 non-null      object
 1   ingredients  8 non-null      object
 2   recipe       8 non-null      object
dtypes: object(3)
memory usage: 324.0+ bytes
None


,name,ingredients,recipe
0,고기와 싸먹는 양파절임만들기 ^^,"표고버섯 5개,양파 1/4개,당근 조금,대파 1/2개,청양고추 2개,식용유 2큰술,...",1. 우선 사온 표고버섯은 밑둥을 제거합니다.\n2. 그리고 얇게 채썰어줍니다.\n...
1,고기와 싸먹는 양파절임만들기 ^^,"식빵 3장,계란 2개,우유 100-120ml,건포도(건크랜베리),슬라이스아몬드 (생...",1. 식빵은 적당한 크기로 깍뚝썰기해서 준비해주세요\n2. 오픈가능한 용기에 무심한...
2,고기와 싸먹는 양파절임만들기 ^^,"무 300g,들기름 2 큰술,대파 1/2 대,멸치 다시마 육수 1종이컵,소금 약간,...",1. 무는 얇게 썰어서 준비해요. ​ * 너무 두껍지 않게 썰도록 해요 :)\n...
3,고기와 싸먹는 양파절임만들기 ^^,"깻잎 10장,참치 1캔,양파 20g,당근 20g,후추 적당히,밥 1공기,된장 1T,...",1. 참치 기름을 숟가락으로 눌러 한껏 빼주고 체에 걸러줍니다. 후추 톡톡 살짝 간...
4,고기와 싸먹는 양파절임만들기 ^^,"돼지고기 600g,김치 600g,양파 1개,대파 1개,설탕 1큰술,식용유,고추장 2...",1. 분량의 양념 재료를 잘 섞어서 준비합니다 김치가 들어가니 양념은 약간 달달한 ...
5,고기와 싸먹는 양파절임만들기 ^^,"양파 1/2개,느타리버섯 50g ,깻잎 30장,부침가루 1/2컵,물 1/2컵,달걀2...",1. 양파는 채썰어주시고 깻잎도 돌돌말아 채썰어주세요 느타리버섯은 손으로 찢어 준...
6,고기와 싸먹는 양파절임만들기 ^^,"스팸 100g,호박 1/3개,계란 2개,후추 약간",1. 스팸 100g을 곱게 채썰어 줍니다. 쥬키니호박 1/3을 곱게 채썰어 줍니다....
7,고기와 싸먹는 양파절임만들기 ^^,"캔골뱅이 1캔,양파 1/4개,대파 1주먹 (or 파채),골뱅이 캔 국물 4스푼,간장...",1. 양파와 파는 얇게 채썰어 찬물에 담가 아린맛을 제거해요\n2. 집에서 파채 ...
